## This compares to DoubleBracketIteration whenever possible

In [1]:
import inspect
import sys
sys.path.append("../../tests")
from test_models_dbi import *
def print_function_source_code( func ):
    out = inspect.getsourcelines(func)  
    from functools import reduce
    print(reduce(str.__add__, out[0]))

In [2]:
import qibo
backend = qibo.backends.construct_backend("numpy")
qibo.set_backend("numpy")
nqubits = 3

[Qibo 0.2.7|INFO|2024-04-26 11:35:38]: Using numpy backend on /CPU:0


In [8]:
def test_gci_implementations_normal_and_oracles(backend,nqubits):

    h_x = SymbolicHamiltonian(
            symbols.X(0)
        + symbols.Z(0) * symbols.X(1)
        + symbols.Y(2)
        + symbols.Y(1) * symbols.Y(2),
        nqubits=3,
    )
    d_0 = SymbolicHamiltonian(symbols.Z(0), nqubits=3)
    h_input = h_x + d_0

    dbi = DoubleBracketIteration(deepcopy(h_input.dense))

    evolution_oracle = EvolutionOracle(h_input, "ZX", mode_evolution_oracle = EvolutionOracleType.numerical)   
    gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle ))

    evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle = EvolutionOracleType.numerical)

    from numpy.linalg import norm

    times = np.linspace(1e-5,1,5)
    for r in times:

        dbi(r, d = d_0.dense.matrix, mode = DoubleBracketGeneratorType.group_commutator )
        k_r = dbi.h.matrix
        dbi.h = deepcopy(h_input.dense) 
        gci(r, diagonal_association= evolution_oracle_diagonal_target )
        j_r = gci.h.matrix
        gci.h = deepcopy(h_input.dense) 
        gci.iterated_hamiltonian_evolution_oracle = deepcopy(evolution_oracle) 

        assert norm(k_r-j_r) < 1e-12

    r = 1
    for _ in range(3):
        dbi(r, d = d_0.dense.matrix, mode = DoubleBracketGeneratorType.group_commutator )
        gci(r, diagonal_association= evolution_oracle_diagonal_target )
        k_r = dbi.h.matrix
        j_r = gci.h.matrix
        
        print(norm(k_r-j_r))
        print(norm(dbi.sigma(k_r)))
        print(norm(dbi.sigma(j_r)))
        assert norm(k_r-j_r) < 1e-12 
        
    evolution_oracle = EvolutionOracle(h_input, "ZX", mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)   
    gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle ))

    evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)
    r = 1
    for _ in range(3):
        dbi(r, d = d_0.dense.matrix, mode = DoubleBracketGeneratorType.group_commutator )
        gci(r, diagonal_association= evolution_oracle_diagonal_target )
        k_r = dbi.h.matrix
        j_r = gci.h.matrix
        
        print(norm(k_r-j_r))
        print(norm(dbi.sigma(k_r)))
        print(norm(dbi.sigma(j_r)))
        assert norm(k_r-j_r) < 1e-12 
        
      

test_gci_implementations_normal_and_oracles(backend,nqubits)

[Qibo 0.2.7|WARNING|2024-04-26 11:42:50]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-04-26 11:42:50]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


5.7435671769611e-15
6.019125649086664
6.019125649086664
2.1112854320411976e-14
6.251889133927085
6.251889133927082
6.133971102219123e-14
6.153453052674688
6.153453052674681
9.354334563414852
6.17137286905534
6.0399303182680475


AssertionError: 

# Check the numerical mode of evolution oracles

This is testing the following:

`dbi_exact` runs $V_{exact} = e^{-sW}$ and rotates $H_1 = V_{exact}^\dagger H_0 V_{exact}$.

`dbi_GC` runs $V_{GC} = GC$ and rotates $K_1 = V_{GC}^\dagger H_0 V_{GC}$.

We assert that dbi_exact and dbi_GC should be within the approximation bound of the GC
$$||J_1-H_1||\le2 ||H_0||\,||R-V||\le C ||H_0|| s^{3/2}$$

`gci` runs $V_{EO,GC} = GC$ and rotates $J_1 = V_{EO,GC}^\dagger H_0 V_{EO,GC}$.

We assert that gci and dbi2 should be within machine precision for the correct sorting.
$$||J_1-K_1||\le2 ||H_0||\,||R-Q||\le \epsilon$$




In [4]:
print_function_source_code(test_gci_evolution_oracles_types_numerical)

def test_gci_evolution_oracles_types_numerical(nqubits,backend,t_step, eps):
    """

    This is testing the following:

    `dbi_exact` runs $V_{exact} = e^{-sW}$ and rotates $H_1 = V_{exact}^\dagger H_0 V_{exact}$.

    `dbi_GC` runs $V_{GC} = GC$ and rotates $K_1 = V_{GC}^\dagger H_0 V_{GC}$.

    We assert that dbi_exact and dbi_GC should be within the approximation bound of the GC
    $$||J_1-H_1||\le2 ||H_0||\,||R-V||\le C ||H_0|| s^{3/2}$$

    `gci` runs $V_{EO,GC} = GC$ and rotates $J_1 = V_{EO,GC}^\dagger H_0 V_{EO,GC}$.

    We assert that gci and dbi2 should be within machine precision for the correct sorting.
    $$||J_1-K_1||\le2 ||H_0||\,||R-Q||\le \epsilon$$
    """

    from numpy.linalg import norm

    h_x = SymbolicHamiltonian(
        symbols.X(0)
        + symbols.Z(0) * symbols.X(1)
        + symbols.Y(2)
        + symbols.Y(1) * symbols.Y(2),
        nqubits=3,
    )
    d_0 = SymbolicHamiltonian(symbols.Z(0), nqubits=3)
    h_input = h_x + d_0

    dbi = Doubl

In [5]:
t_step =0.01
eps = 1e-2
test_gci_evolution_oracles_types_numerical(nqubits,backend,t_step, eps)



[Qibo 0.2.7|WARNING|2024-04-26 11:35:39]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-04-26 11:35:39]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-04-26 11:35:39]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


AssertionError: 

## Check numerical GCI with evolution oracles against `DoubleBracketIteration`

Basically we have again a bound for how $H_k$ differs from $J_k$ for various duriations of the DBR.

The plots below show that the GCI implementation for 1 step agrees precisely

In [ ]:
h_x = SymbolicHamiltonian(
        symbols.X(0)
    + symbols.Z(0) * symbols.X(1)
    + symbols.Y(2)
    + symbols.Y(1) * symbols.Y(2),
    nqubits=3,
)
d_0 = SymbolicHamiltonian(symbols.Z(0), nqubits=3)
h_input = h_x + d_0

dbi = DoubleBracketIteration(deepcopy(h_input.dense))
dbi.mode = DoubleBracketGeneratorType.single_commutator

evolution_oracle = EvolutionOracle(h_input, "ZX",
                        mode_evolution_oracle = EvolutionOracleType.numerical)    

evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
           mode_evolution_oracle = EvolutionOracleType.numerical)

gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle ))


import matplotlib.pyplot as plt
from numpy.linalg import norm
norms  = []
norms2  = []
norms_bound = []
times = np.linspace(1e-5,1,30)
for r in times:
    dbi(r, d = d_0.dense.matrix, mode = DoubleBracketGeneratorType.single_commutator )
    h_r = dbi.h.matrix
    dbi.h = deepcopy(h_input.dense) 
    dbi(r, d = d_0.dense.matrix, mode = DoubleBracketGeneratorType.group_commutator )
    k_r = dbi.h.matrix
    dbi.h = deepcopy(h_input.dense) 
    gci(r, diagonal_association= evolution_oracle_diagonal_target )
    j_r = gci.h.matrix
    gci.h = deepcopy(h_input.dense) 
    
    w = dbi.commutator(h_input.dense.matrix,d_0.dense.matrix)
    norms_bound.append(norm(h_input.dense.matrix)*r**1.48 * (
        np.linalg.norm(dbi.commutator(h_input.dense.matrix,w)) + np.linalg.norm(dbi.commutator(d_0.matrix,w))
    ))
   
    norms.append(norm(h_r -k_r ))
    norms2.append(norm(h_r-j_r))
    
plt.loglog(times, norms_bound)
plt.loglog(times,norms, 
          '-s')
plt.loglog(times,norms2, '-x')

### This is captured in the following test
(we restrict to both group commutator modes as for more steps there will be deviations)

In [ ]:
  

    v_exact = dbi.eval_dbr_unitary(t_step, d=d_0.dense.matrix, mode=DoubleBracketGeneratorType.single_commutator)
    v_gc = dbi.eval_dbr_unitary(t_step, d=d_0.dense.matrix, mode=DoubleBracketGeneratorType.group_commutator)

    assert norm(v_exact - v_gc) < norms_bound

    evolution_oracle = EvolutionOracle(h_input, "ZX",
                        mode_evolution_oracle = EvolutionOracleType.numerical)    
    d_02 = SymbolicHamiltonian(symbols.Z(0), nqubits=3)
    evolution_oracle_diagonal_target =  EvolutionOracle(d_02, "D0",
               mode_evolution_oracle = EvolutionOracleType.numerical)

    gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle ))
    #gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator

    u_gc_from_oracles = gci.group_commutator( t_step, evolution_oracle_diagonal_target )   
    u_gci = u_gc_from_oracles['forwards']

    assert norm(u_gci.conj().T - u_gc_from_oracles['backwards']) < 1e-12


    v_exact = dbi.eval_dbr_unitary(t_step, d=d_0.dense.matrix, mode=DoubleBracketGeneratorType.single_commutator)
    w = dbi.commutator(h_input.dense.matrix,d_0.dense.matrix)
    norms_bound = 0.5*t_step**1.48 * (
        np.linalg.norm(dbi.commutator(h_input.dense.matrix,w)) + np.linalg.norm(dbi.commutator(d_0.matrix,w))
    )
    assert norm(v_exact - u_gci) < norms_bound

    gci(t_step, diagonal_association= evolution_oracle_diagonal_target )
    j_1 = gci.iterated_hamiltonian_evolution_oracle.h.matrix
    assert norm(h_1-j_1) < 2 * norm(h_input.dense.matrix) * norms_bound   
    assert norm(j_1-k_1) < 1e-12 

### Next many steps: the `DoubleBracketIteration` implementation agrees with evolution oracles for the gc mode. In the exact commutator exponential a deviation builds up

In [ ]:
r=0.01

norms_it  = []
norms_it2  = []
dbi = DoubleBracketIteration(deepcopy(h_input.dense))
dbi2 = DoubleBracketIteration(deepcopy(h_input.dense))
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle ))
for _ in range(15):
    dbi(r, d = d_0.dense.matrix, mode = DoubleBracketGeneratorType.single_commutator )
    dbi2(r, d = d_0.dense.matrix, mode = DoubleBracketGeneratorType.group_commutator )
    gci(r, diagonal_association= evolution_oracle_diagonal_target )
    h_r = dbi.h.matrix
    k_r = dbi2.h.matrix
    j_r = gci.h.matrix
    norms_it.append(norm(k_r-j_r))
    norms_it2.append(norm(h_r-j_r))    
plt.plot(norms_it)
plt.show()
plt.plot(norms_it2)

## Simple test of group commutator

In [ ]:
def test_exact_dbr_vs_gc_bound(nqubits,backend):
    dbi = DoubleBracketIteration(deepcopy(h_input.dense))
    dbi.mode = DoubleBracketGeneratorType.single_commutator


    dbi2 = DoubleBracketIteration(deepcopy(h_input.dense))
    dbi2.mode = DoubleBracketGeneratorType.group_commutator

    import matplotlib.pyplot as plt
    norms  = []
    for r in np.linspace(1e-5,0.1,30):
        dbi(r, d=d_0.dense.matrix)
        dbi2(r, d=d_0.dense.matrix)
        norms.append(norm(dbi.h.matrix- dbi2.h.matrix))
    
    #plt.plot(np.linspace(1e-5,.1,30), [x**1.5*1000 for x in np.linspace(1e-5,.1,30)])
    plt.plot(np.linspace(1e-5,.1,30),norms)
test_exact_dbr_vs_gc_bound(nqubits,backend)

## Check the GC bound is valid and corectly implemented in DoubleBracketIteration

The bound is $$||e^{-t[D,H]}-GC(s=\sqrt{t})||\le t^{3/2}(||[H,[D,H]||+||[D,[D,H]]||$$


In [ ]:
print_function_source_code(test_double_bracket_iteration_eval_dbr_unitary)

In [ ]:
test_double_bracket_iteration_eval_dbr_unitary(backend, nqubits)

### We repeat the function in order to plot the bound

In [ ]:
r"""The bound is $$||e^{-[D,H]}-GC||\le s^{3/2}(||[H,[D,H]||+||[D,[D,H]]||$$"""
h0 = random_hermitian(2**nqubits, backend=backend)
d = backend.cast(np.diag(np.diag(backend.to_numpy(h0))))
dbi = DoubleBracketIteration(
    Hamiltonian(nqubits, h0, backend=backend),
    mode=DoubleBracketGeneratorType.group_commutator,
)

times = np.linspace(0.001, 0.01, 10)
norms = []
norms_bound = []
for s in times:
    u = dbi.eval_dbr_unitary(
        s, d=d, mode=DoubleBracketGeneratorType.single_commutator
    )
    v = dbi.eval_dbr_unitary(
        s, d=d, mode=DoubleBracketGeneratorType.group_commutator
    )

    norms.append(np.linalg.norm(u - v) )
    w = dbi.commutator(h0,d)
    norms_bound.append(0.5*s**1.48 * (
        np.linalg.norm(dbi.commutator(h0,w)) + np.linalg.norm(dbi.commutator(d,w))
    ))
    assert np.linalg.norm(u - v) < 10 * s**1.49 * (
            np.linalg.norm(h0) + np.linalg.norm(d)
        ) * np.linalg.norm(h0) * np.linalg.norm(d)

Unfortunately we cannot assume that 
`assert popt[0] < 1.51`
because in principle data satisfying the bound can be ragged and could have a large slope.
When the bound is saturated, however, then the following fit works:

In [ ]:
from scipy.optimize import curve_fit
popt, pcov = curve_fit( (lambda x, a, b: a*x+b ), [np.log(t) for t in times], [np.log(n) for n in norms])
assert popt[0] < 1.51

In practice that assertion usually goes through. We check this further by these plots

In [ ]:
import matplotlib.pyplot as plt
plt.plot(norms)
plt.plot(norms_bound)

In [ ]:
plt.loglog(times,norms)
plt.loglog(times,norms_bound)

Usually random Hamiltonian matrices give something $a\approx 1.49$

In [ ]:
popt

# Check the convergence of the Trotter-Suzuki Hamiltonian simulation oracle


In [ ]:
print_function_source_code(test_dbi_evolution_oracle)

In [ ]:
test_dbi_evolution_oracle(backend,nqubits,1 ,1e-2)

This was just a check to see if using the circuit method works. Then covering the numerical and text flag.

In [ ]:
from qibo.models.double_bracket import *

def test_dbi_vs_gci(t_step, eps):
    if t_step > 1:
        t_step = 0.1
        
    h_x = SymbolicHamiltonian( symbols.X(0) + symbols.Z(0) * symbols.X(1) + symbols.Y(2) 
                              + symbols.Y(1) * symbols.Y(2), nqubits = 3 )
    d_0 = SymbolicHamiltonian(symbols.Z(0), nqubits = 3 )
    h_input = h_x + d_0    

    dbi = DoubleBracketIteration(deepcopy(h_input.dense))
    dbi.mode = DoubleBracketGeneratorType.group_commutator
    dbi(t_step, d = d_0.dense.matrix )
    
    dbi2 = DoubleBracketIteration(deepcopy(h_input.dense))
    dbi2.mode = DoubleBracketGeneratorType.single_commutator
    dbi2(t_step, d = d_0.dense.matrix )
    
    evolution_oracle = EvolutionOracle(h_input, "ZX",
                        mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)    
    evolution_oracle.eps_trottersuzuki = eps
    
    evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation)
    
    gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle ))
    gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_other_sorting
    
    assert norm(gci.h.matrix - dbi.h.matrix) < 1e-12   
    assert norm(gci.h.matrix - dbi2.h.matrix) < 20 * norm(h_input) * t_step
    
    dbi(t_step, d = d_0.dense.matrix )    
    dbi2(t_step, d = d_0.dense.matrix )    
    gci(t_step, diagonal_association=evolution_oracle_diagonal_target)  
    assert norm(gci.h.matrix - dbi.h.matrix) < 1e-12   
    assert norm(gci.h.matrix - dbi2.h.matrix) < (20 * norm(h_input) * t_step)**2
    
    
test_dbi_vs_gci(.1, 1e-5)

# Show that double bracket iteration  and gci gc and gc_reduced converge for small s BHMM




In [ ]:
dbi(t_step, d = d_0.dense.matrix)


#### 2. Evolution oracle hamiltonian simulation


In [ ]:
evolution_oracle_hamiltonian_simulation = EvolutionOracle(deepcopy(h_input), "ZX",
                               mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)


In [ ]:
evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation)



In [ ]:
t_step = 1
def check_hs_eps(eps):
    evolution_oracle_hamiltonian_simulation.eps_trottersuzuki = eps
    U_hamiltonian_simulation = evolution_oracle_hamiltonian_simulation.circuit(t_step).unitary()
    V_target = h_input.exp(t_step)
    print(eps,norm(U_hamiltonian_simulation-V_target))

In [ ]:
check_hs_eps(0.1)

In [ ]:
check_hs_eps(.001)

In [ ]:
check_hs_eps(1e-4)

In [ ]:

gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle_hamiltonian_simulation ))
gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_other_sorting


In [ ]:
gci.iterated_hamiltonian_evolution_oracle.h

In [ ]:
unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

In [ ]:
for a in DoubleBracketRotationType:
    print(a.name)

In [ ]:
def test_gc_numerical_vs_circuit(eps):
    gci.iterated_hamiltonian_evolution_oracle.eps_trottersuzuki = eps
    evolution_oracle_diagonal_target.eps_trottersuzuki = eps
    
    gci.iterated_hamiltonian_evolution_oracle.please_be_verbose = False
    evolution_oracle_diagonal_target.please_be_verbose = False
    

    gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_reduced

    gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
    evolution_oracle_diagonal_target.mode_evolution_oracle  = EvolutionOracleType.numerical

    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

    unitary_gc_existing = wrapper_gc(dbi, np.sqrt(t_step),d_0.dense.matrix)

    bckwd = unitary_gc_from_oracles['backwards']
    fwd = unitary_gc_from_oracles['forwards']

    e1 = gci.iterated_hamiltonian_evolution_oracle.circuit(np.sqrt(t_step))
    e2 = evolution_oracle_diagonal_target.circuit(np.sqrt(t_step))

    gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
    evolution_oracle_diagonal_target.mode_evolution_oracle  = EvolutionOracleType.hamiltonian_simulation

    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association = evolution_oracle_diagonal_target )

    unitary_gc_existing = wrapper_gc(dbi, np.sqrt(t_step),d_0.dense.matrix)

    bckwdhs = unitary_gc_from_oracles['backwards'].unitary()
    fwdhs = unitary_gc_from_oracles['forwards'].unitary()

    e1hs = gci.iterated_hamiltonian_evolution_oracle.circuit(np.sqrt(t_step)).unitary()
    e2hs = evolution_oracle_diagonal_target.circuit(np.sqrt(t_step)).unitary()

    print("e1:", norm( e1hs-e1))
    print("e2:", norm(e2hs-e2))
    print("Testing inversion forwards:", norm(bckwd-bckwdhs))
    print("Testing inversion backwards:", norm(fwd - fwdhs))

test_gc_numerical_vs_circuit(0.1)

test_gc_numerical_vs_circuit(0.0001)

In [ ]:
def test_group_commutator_against_itself(gci, evolution_diagonal,eps = 0.0001):
    gci.iterated_hamiltonian_evolution_oracle.eps_trottersuzuki = eps
    evolution_oracle_diagonal_target.eps_trottersuzuki = eps
    for s in np.linspace(0,1,5):
        gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
        evolution_diagonal.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
        gc_hs = gci.group_commutator(s,evolution_diagonal)
        gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
        evolution_diagonal.mode_evolution_oracle = EvolutionOracleType.numerical
        gc_np = gci.group_commutator(s,evolution_diagonal)
        print(norm(gc_np['forwards']-gc_hs['forwards'].unitary()))
        print(norm(gc_np['backwards']-gc_hs['backwards'].unitary()))

gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator        
test_group_commutator_against_itself(gci, evolution_oracle_diagonal_target)

gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_other_sorting        
test_group_commutator_against_itself(gci, evolution_oracle_diagonal_target)

gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_reduced      
test_group_commutator_against_itself(gci, evolution_oracle_diagonal_target)

In [ ]:
def test_gc_numerical_vs_bracket(eps = 0.3):
    gci.iterated_hamiltonian_evolution_oracle.eps_trottersuzuki = eps
    evolution_oracle_diagonal_target.eps_trottersuzuki = eps
    
    gci.iterated_hamiltonian_evolution_oracle.please_be_verbose = False
    evolution_oracle_diagonal_target.please_be_verbose = False
    

    gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator

    gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
    evolution_oracle_diagonal_target.mode_evolution_oracle  = EvolutionOracleType.numerical

    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

    import scipy
    unitary_gc_existing1 = scipy.linalg.expm( t_step * (d_0.dense.matrix @ h_x.dense.matrix
                                         -h_x.dense.matrix@d_0.dense.matrix))

    bckwd = unitary_gc_from_oracles['backwards']
    fwd = unitary_gc_from_oracles['forwards']

    e1 = gci.iterated_hamiltonian_evolution_oracle.circuit(np.sqrt(t_step))
    e2 = evolution_oracle_diagonal_target.circuit(np.sqrt(t_step))

    gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
    evolution_oracle_diagonal_target.mode_evolution_oracle  = EvolutionOracleType.hamiltonian_simulation

    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

    unitary_gc_existing = wrapper_gc(dbi, np.sqrt(t_step),-d_0.dense.matrix)

    bckwdhs = unitary_gc_from_oracles['backwards'].unitary()
    fwdhs = unitary_gc_from_oracles['forwards'].unitary()

    e1hs = gci.iterated_hamiltonian_evolution_oracle.circuit(np.sqrt(t_step)).unitary()
    e2hs = evolution_oracle_diagonal_target.circuit(np.sqrt(t_step)).unitary()
    print("Test:", norm(unitary_gc_existing-unitary_gc_existing1))
    print("e1:", norm( e1hs-e1))
    print("e2:", norm(e2hs-e2))
    print("Testing inversion forwards:", norm(bckwd-bckwdhs))
    print("Testing inversion backwards:", norm(fwd - fwdhs))

test_gc_numerical_vs_bracket(0.1)

test_gc_numerical_vs_bracket(0.0001)

In [ ]:
def gc_hs_eps(eps):
    gci.iterated_hamiltonian_evolution_oracle.eps_trottersuzuki = eps
    evolution_oracle_diagonal_target.eps_trottersuzuki = eps
    
    gci.iterated_hamiltonian_evolution_oracle.please_be_verbose = True
    evolution_oracle_diagonal_target.please_be_verbose = False
    
    gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
    evolution_oracle_diagonal_target.mode_evolution_oracle  = EvolutionOracleType.numerical

    gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator
    
    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
               diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )
    
    unitary_gc_existing = wrapper_gc(dbi, np.sqrt(t_step),-d_0.dense.matrix)
    
    bckwd = unitary_gc_from_oracles['backwards']
    fwd = unitary_gc_from_oracles['forwards']
    
    e1 = gci.iterated_hamiltonian_evolution_oracle.circuit(np.sqrt(t_step))
    e2 = evolution_oracle_diagonal_target.circuit(np.sqrt(t_step))
    
    print("Backwards:", norm( bckwd - unitary_gc_existing))
    print("Forwards:", norm(fwd - unitary_gc_existing))
    print("Testing inversion forwards:", norm(fwd - e2.conj().T @ e1.conj().T @ e2 @e1))
    print("Testing inversion backwards:", norm(bckwd - e2.conj().T @ e1.conj().T @ e2 @e1))
    print("Testing inversion forwards:", norm(fwd - e1.conj().T @ e2.conj().T @  e1 @e2))
    print("Testing inversion backwards:", norm(bckwd - e1.conj().T @ e2.conj().T @  e1 @e2))
    print("Testing inversion forwards:", norm(fwd - e1@e2@ e1.conj().T @ e2.conj().T ))
    print("Testing inversion backwards:", norm(bckwd - e1@e2@e1.conj().T @ e2.conj().T ))
    print("Testing inversion forwards:", norm(fwd - e1.conj().T@e2@ e1 @ e2.conj().T ))
    print("Testing inversion backwards:", norm(bckwd - e1.conj().T@e2@e1 @ e2.conj().T ))
    print("Testing reversal:", norm(bckwd@fwd - unitary_gc_existing @unitary_gc_existing.T.conj() ))
    print("Testing reversal:", norm(bckwd@fwd - e1@e1.conj().T ))

In [ ]:
gc_hs_eps(0.1)

We may improve the discrepancy by setting smaller eps

In [ ]:
gc_hs_eps(0.0001)

In [ ]:
norm(unitary_gc_from_oracles['forwards'].unitary() - unitary_gc_existing)

In [ ]:
norm(unitary_gc_from_oracles['backwards'].unitary() - unitary_gc_existing)

In [ ]:
stop

#### Test more explicitly


In [ ]:
u_h = gci.iterated_hamiltonian_evolution_oracle.circuit( np.sqrt(t_step)).unitary()
u_d = evolution_oracle_diagonal_target.circuit( np.sqrt(t_step)).unitary()
u_h_reversed = gci.iterated_hamiltonian_evolution_oracle.circuit( -np.sqrt(t_step)).unitary()
u_d_reversed = evolution_oracle_diagonal_target.circuit( -np.sqrt(t_step)).unitary()
norm( u_h_reversed @ u_d_reversed @ u_h @ u_d - unitary_gc_existing)

In [ ]:
u_h = gci.iterated_hamiltonian_evolution_oracle.circuit( np.sqrt(t_step))
u_d = evolution_oracle_diagonal_target.circuit( np.sqrt(t_step))
u_h_reversed = gci.iterated_hamiltonian_evolution_oracle.circuit( -np.sqrt(t_step))
u_d_reversed = evolution_oracle_diagonal_target.circuit( -np.sqrt(t_step))
norm( (u_h_reversed + u_d_reversed + u_h + u_d).unitary() - unitary_gc_existing)

#### 3. Evolution oracle numpy


In [ ]:
evolution_oracle_numerical = EvolutionOracle(deepcopy(h_input), "ZX",
                               mode_evolution_oracle = EvolutionOracleType.numerical)

gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle_numerical ))

In [ ]:
evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle=EvolutionOracleType.numerical)


In [ ]:
unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

Compared to the group commutator using Hamiltonian simulation there will be small deviations that arise from Trotter-Suzuki decomposition

In [ ]:
norm(unitary_gc_from_oracles['backwards'] - unitary_gc_existing)

In [ ]:
norm(unitary_gc_from_oracles['forwards'] - unitary_gc_existing)

We may check by switching the group commutator flag that the difference comes from ordering and inversions

In [ ]:
gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_other_sorting

unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )
norm(unitary_gc_from_oracles['forwards'] - unitary_gc_existing)

#### 4. Check gci rotation

In [ ]:
evolution_oracle_hamiltonian_simulation = EvolutionOracle(deepcopy(h_input), "ZX",
                               mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)
evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation)
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle_hamiltonian_simulation ))

In [ ]:
type(gci.iterated_hamiltonian_evolution_oracle)

In [ ]:
gci(t_step, diagonal_association=evolution_oracle_diagonal_target)

In [ ]:
type(gci.iterated_hamiltonian_evolution_oracle)

In [ ]:
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle

In [ ]:
u_frame_shifted = gci.iterated_hamiltonian_evolution_oracle.circuit(t_step).unitary()

In [ ]:
norm( dbi.h.exp(t_step) - u_frame_shifted)

In [ ]:
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle


In [ ]:
gci(t_step, diagonal_association=evolution_oracle_diagonal_target)

In [ ]:
dbi(t_step, d = evolution_oracle_diagonal_target.h.dense.matrix)

In [ ]:
u_frame_shifted = gci.iterated_hamiltonian_evolution_oracle.circuit(t_step).unitary()


In [ ]:
norm( dbi.h.exp(t_step) - u_frame_shifted)

In [ ]:
for k in range(3):
    gci(t_step, diagonal_association=evolution_oracle_diagonal_target)
    dbi(t_step, d = evolution_oracle_diagonal_target.h.dense.matrix)
    print(norm( dbi.h.exp(t_step) - u_frame_shifted))

In [ ]:
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation

In [ ]:
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
gc_numpy = gci.group_commutator( np.sqrt(t_step),
                                             diagonal_association_evolution_oracle= EvolutionOracle(d_0, "D0",mode_evolution_oracle=EvolutionOracleType.numerical))

In [ ]:
## Test more fancy functionalities
input_hamiltonian_evolution_oracle_hamiltonian_simulation.please_be_verbose = False
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(input_hamiltonian_evolution_oracle_hamiltonian_simulation ))
d_ev =  EvolutionOracle(d_0, "D0",mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation)

gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
query_list = gci.group_commutator( np.sqrt(t_step),
                                             diagonal_association_evolution_oracle= d_ev )

norm(query_list['forwards'].unitary() -query_list['backwards'].unitary().conj().T)




In [ ]:
norm(query_list['forwards'] -query_list['backwards'].T.conj())

In [ ]:
#Test file entry
u = gci.iterated_hamiltonian_evolution_oracle.circuit( np.sqrt(t_step)).unitary()
u2 = gci.iterated_hamiltonian_evolution_oracle.circuit( -np.sqrt(t_step)).unitary()
norm(u-u2.T.conj())

In [ ]:
d_0.mode_evolution_oracle = EvolutionOracleType.text_strings
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.text_strings
query_list = gci.group_commutator( np.sqrt(t_step*2),
             diagonal_association_evolution_oracle= EvolutionOracle(d_0, "D0"))


query_list['forwards']